In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from keras.utils import to_categorical
from kerastuner.tuners import RandomSearch
import tensorflow as tf
from keras.utils import plot_model
import keras.backend as K
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
train_features = pd.read_csv("/kaggle/input/lish-moa/train_features.csv")
train_targets_scored = pd.read_csv("/kaggle/input/lish-moa/train_targets_scored.csv")
train_targets_unscored = pd.read_csv("/kaggle/input/lish-moa/train_targets_nonscored.csv")
test_features = pd.read_csv("/kaggle/input/lish-moa/test_features.csv")

sample_submission = pd.read_csv("/kaggle/input/lish-moa/sample_submission.csv")

In [ ]:
train_features.head()

In [ ]:
train_features=pd.get_dummies(train_features,columns=['cp_type','cp_dose'])
test_features=pd.get_dummies(test_features,columns=['cp_type','cp_dose'])

In [ ]:
del train_features['sig_id']
del train_targets_scored['sig_id']
del test_features['sig_id']

In [ ]:
def build_model():
  K.clear_session()
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Input(shape=(train_features.shape[1],)))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dense(units=1024,activation='relu'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(units=2048,activation='relu'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(206, activation="sigmoid"))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy","binary_crossentropy"])
  return model

In [ ]:
train_features

In [ ]:
np.array(train_targets_scored)

In [ ]:
pred = np.zeros((test_features.shape[0], 206))

n_split = 5
kfoldnumber= 0
for train_index, validation_index in KFold(n_split).split(train_features.values):
    kfoldnumber += 1 
    print('Fold number: ',kfoldnumber)
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')
    x_train,x_validation=train_features.values[train_index],train_features.values[validation_index]
    y_train,y_validation=train_targets_scored.values[train_index],train_targets_scored.values[validation_index]
    mod = build_model()
    mod.fit(x_train,
          y_train,
    validation_data=(np.array(x_validation),np.array(y_validation)),
    shuffle=True,
    epochs=75,verbose=0,callbacks=[reduce_lr_loss])
    #print('train: ',  mod.evaluate(x_train[train_index], y_train[train_index], verbose=0, batch_size=128))
    #print('validation: ',  mod.evaluate(x_train[validation_index], y_train[validation_index], verbose=0, batch_size=128))
    pred += mod.predict(test_features, batch_size=128, verbose=0) / n_split    

## COMPILE SUBMISSION

In [ ]:
sub = pd.DataFrame(data=pred, columns=train_targets_scored.columns)
sub.insert(0, column = 'sig_id', value=sample_submission['sig_id'])
sub.to_csv('submission.csv',index=False)

In [ ]:
#subconcat= pd.concat([sub,test_features.cp_type_ctl_vehicle],axis=1)
#for rows in range(len(subconcat)):
#    if subconcat.cp_type_ctl_vehicle[rows] == 1:
#        subconcat.iloc[rows] = 0
        #
#subconcat.insert(0, column = 'sig_id', value=sample_submission['sig_id'])
#subconcat.to_csv('submission.csv',index=False)